# Plotting Sparsity Plots

In [ ]:
import numpy as np
import scipy as sp
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import copy
sns.set_style("white")

In [ ]:
#load genetics dataset
with open('./data/gen_data', 'rb') as handle:
    gen_train = pickle.load(handle)

N = gen_train['N']
D = gen_train['D']
y = np.int8(gen_train['y'].reshape(N,))
x = gen_train['x'].values

In [ ]:
#Calculate correlations between covariates
corX = np.corrcoef(np.transpose(x))
corind = np.meshgrid(np.arange(0,D),np.arange(0,D))

In [ ]:
#Plot correlations between covariates
f = plt.figure(figsize = (4,4))
im=plt.imshow(corX,origin = 'lower',cmap = 'coolwarm')
cbar = f.colorbar(im)
cbar.ax.tick_params(labelsize=16)
plt.xlabel('d',fontsize = 18)
plt.ylabel('d',fontsize = 18)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
f.savefig('../../../Writeups/ICML 2019/figures/correlations_small.pdf', bbox_inches='tight')

In [ ]:
#Load beta samples
with open('./parameters/par_bb_logreg_gen_T450_a1_ARD_B4000_small', 'rb') as handle:
    gen_graph = pickle.load(handle)

beta_samps = gen_graph['beta'][:,:,0:50]
a_samps = gen_graph['a']
b_samps = gen_graph['b']

In [ ]:
#Calculate medians and credible intervals
from scipy.stats import mode

mean_beta = np.mean(beta_samps,axis = 1)
#median_beta = np.median(beta_samps,axis = 1)
median_beta = np.percentile(beta_samps,50,axis = 1,interpolation ='linear')


lower10 = np.percentile(beta_samps,10,axis = 1,interpolation ='linear')
upper10 = np.percentile(beta_samps,90,axis = 1,interpolation ='linear')

lower125 = np.percentile(beta_samps,12.5,axis = 1,interpolation ='linear')
upper125 = np.percentile(beta_samps,87.5,axis = 1,interpolation ='linear')

lower15 = np.percentile(beta_samps,15,axis = 1,interpolation ='linear')
upper15 = np.percentile(beta_samps,85,axis = 1,interpolation ='linear')

lower25 = np.percentile(beta_samps,2.5,axis = 1,interpolation ='linear')
upper25 = np.percentile(beta_samps,97.5,axis = 1,interpolation ='linear')

In [ ]:
f=plt.figure(figsize=(12,6))
ind = np.array([10,14,24,31,37])-1
start= 0
end = 370

for i in range(D):
    if i in ind:
        sns.lineplot(np.log(b_samps[start:end]/a_samps[start:end]), (median_beta[start:end,i]),label = r'$\beta_{{{}}}$'.format(i+1))
        plt.fill_between(np.log(b_samps[start:end]/a_samps[start:end]),lower10[start:end,i], upper10[start:end,i], alpha = 0.3)
    #else:
        #sns.lineplot(np.log(b_samps[start:end]/a_samps[start:end]), median_beta[start:end,i],color = 'grey')
plt.xlabel('log(c)',fontsize = 24)
plt.ylabel('Posterior median',fontsize = 24)
plt.legend( fontsize = 14, loc = 'upper left')
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)

In [ ]:
f=plt.figure(figsize=(12,6))
ind = np.array([10,14,24,31,37])-1
start= 0
end = 400

for i in range(D):
    if i in ind:
        sns.lineplot(np.log(b_samps[start:end]/a_samps[start:end]), (np.abs(median_beta[start:end,i])),label = r'$\beta_{{{}}}$'.format(i+1))
        #plt.fill_between(np.log(b_samps[start:end]/a_samps[start:end]),lower10[start:end,i], upper10[start:end,i], alpha = 0.3)
    else:
        sns.lineplot(np.log(b_samps[start:end]/a_samps[start:end]), np.abs(median_beta[start:end,i]),color = 'grey')
plt.xlabel('log(c)',fontsize = 24)
plt.ylabel('Posterior median',fontsize = 24)
plt.legend( fontsize = 14, loc = 'upper left')
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)

In [ ]:
#Calculate KDE for beta_108
from scipy.stats import gaussian_kde
Tstart=80
Tend = 340
comp = 13
xrange = np.arange(-0.1,1,0.001)
nind = np.size(xrange)
kde = np.zeros((Tend-Tstart,nind))
for i in range(Tend-Tstart):
    m = gaussian_kde(beta_samps[i+Tstart,:,comp])
    kde[i] = m.evaluate(xrange)

In [ ]:
#Plot KDE
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib import cm
from matplotlib.gridspec import GridSpec
gs=GridSpec(1,8)
Trange = np.log(b_samps[Tstart:Tend]/a_samps[Tstart:Tend])
grid = np.meshgrid(xrange,Trange)
fig = plt.figure(figsize = (18,8))
ax = fig.add_subplot(gs[0,0:5], projection='3d')


surf = ax.plot_surface(grid[0], grid[1], kde, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax.set_xlabel(r'$\beta_{{{}}}$'.format(comp+1),fontsize = '24',labelpad = 19)
ax.set_ylabel(r'$\log(c)$',fontsize = '24',labelpad = 19)
ax.set_zlabel('Posterior Density',fontsize = '18',labelpad = 15)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax.zaxis.set_tick_params(labelsize=16)
ax.view_init(70, 80)
#cbar = fig.colorbar(surf)
#cbar.ax.tick_params(labelsize=16)
plt.gca().invert_xaxis()


aspect = 1.1*np.shape(kde)[1]/np.shape(kde)[0]
plt.subplot(gs[0,5:8])
im=plt.imshow(kde,origin = 'lowerright',cmap = 'coolwarm',aspect = 1/aspect,extent = [-0.1,1,Trange[0],Trange[Tend-Tstart-1]])
cbar = fig.colorbar(im)
cbar.ax.tick_params(labelsize=16)
plt.xlabel(r'$\beta_{14}$',fontsize = 18)
plt.ylabel(r'$\log(c)$',fontsize = 18, rotation=270,labelpad = 27)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)